<a id="section_CART"></a> 
## Workshop Final DS Digital House

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
pd.set_option('display.max_columns', None)
import missingno as msno
#from dataprep.eda import plot, plot_correlation, create_report, plot_missing

%matplotlib inline

## Importando os dados

In [ ]:
# importando bases de treino e teste
df_test = pd.read_csv(r"../DataSet/test.csv", low_memory = False)
df_train = pd.read_csv(r"../DataSet/train.csv", low_memory = False)

## Agrupando os datasets para limpar os dados

In [ ]:
# incluindo colunas para sperar os dados da mesma maneira que estavam originalmente
df_train['test'] = 0
df_test['test'] = 1

# incluindo coluna de score com nan nos dados de teste
df_test['Credit_Score'] = np.nan

In [ ]:
# concatenando os datasets ja que eles tem as mesmas colunas

df_total = pd.concat([df_train, df_test], ignore_index = True)

In [ ]:
print('dados de treino: ', df_train.shape)
print('dados de teste: ', df_test.shape)
print('todos os dados agrupados: ', df_total.shape)

## Colunas dataset:

* ID - Identificador de entrada
* Customer_ID - ID cliente
* Month - Mês do ano
* Name - nome do cliente
* Age - Idade Cliente
* SSN - Social Security Number (CPF no Brasil)
* Occupation - Ocupação do cliente
* Annual_Income - renda anual
* Monthly_Inhand_Salary - Salario mensal do cliente
* Num_Bank_Accounts - quantidade de contas em bancos
* Num_Credit_Card - quantidade de cartões de crédito
* Interest_Rate - taxa de juros cartão de crédito
* Num_of_Loan - Quantidade de empréstimos feitos no banco
* Type_of_Loan - tipo de empréstimo feito pelo cliente
* Delay_from_due_date - qtd. de dias de atraso pagamento cartão
* Num_of_Delayed_Payment - Média de pagamentos atrasado pelo cliente
* Changed_Credit_Limit - Variação percentual de limite do cartão de crédito
* Num_Credit_Inquiries - Quantidade de "cobranças" no cartão
* Credit_Mix - mix de crédito do cliente
* Outstanding_Debt - restante à ser pago da dívida
* Credit_Utilization_Ratio - Taxa de utlização do cartão de crédito
* Credit_History_Age - Tempo de histórico de crédito do cliente
* Payment_of_Min_Amount - Pagamento minimo
* Total_EMI_per_month - Pagamento fixo em dolares por mes
* Amount_invested_monthly - Quantidade de dinheiro investido pelo cliente mensalmente
* Payment_Behaviour - Comportamento de pagamento cliente
* Monthly_Balance - Saldo Mensal Cliente
* Credit_Score - Target
* test - coluna utilizada para separar o dataset nos dados de treino e teste

## Checando os principais valores de algumas colunas

In [ ]:
colunas = df_total.columns

for coluna in colunas:
    print('Variavel: ', coluna)
    print(20*'-')
    print(df_total[coluna].value_counts(dropna=False))

### Observações

1. Colunas numéricas com "_" ok
    * Age,
    * Annual_Income,
    * Monthly_Inhand_Salary,
    * Num_Bank_Accounts,
    * Num_Credit_Card,
    * Interest_Rate
    * Num_of_Loan
    * Delay_from_due_date
    * Num_of_Delayed_Payment
    * Changed_Credit_Limit
    * Num_Credit_Inquiries
    * Outstanding_Debt
    * Credit_Utilization_Ratio
    * Total_EMI_per_month
    * Amount_invested_monthly
    * Monthly_Balance
2. SSN #F%$D@*&8 ok
3. Occupation _______ ok
4. Type_of_Loan - transformar em lista e indexar
5. Changed_Credit_Limit "_" -> NaN ok
6. Credit_Mix "_" -> NaN ok
7. Credit_History_Age Transformar em qtd. Meses
8. Payment_of_Min_Amount "NM" -> NaN ok
9. Payment_Behaviour "!@9#%8" -> NaN e transformar dado ok

## ajustando os campos númericos que estão definidos como string por terem underlines em alguns registros

In [ ]:
# Campos númericos que estão como string - retirar underline dos numeros

colunas_ul = ['Age', 'Annual_Income', 'Num_of_Loan', 'Num_of_Delayed_Payment',
              'Changed_Credit_Limit', 'Outstanding_Debt', 'Amount_invested_monthly', 'Monthly_Balance']
for row in colunas_ul:
    df_total[row] = df_total[row].str.replace(r'_+', '')

## Data Wrangling

In [ ]:
# removendo caracter estranho do SSN
df_total['SSN'].replace('#F%$D@*&8', np.NaN, inplace=True)

# removendo os underlines e colocando NaN na coluna Occupation
df_total['Occupation'].replace('_______', np.NaN, inplace=True)

df_total['Changed_Credit_Limit'].replace(['_', ''], np.NaN, inplace=True)

df_total['Credit_Mix'].replace('_', np.NaN, inplace=True)

df_total['Payment_of_Min_Amount'].replace('NM', np.NaN, inplace=True)

df_total['Payment_Behaviour'].replace('!@9#%8', np.NaN, inplace=True)

In [ ]:
# convertendo Credit_History_Age em quantidade de meses
def converter_mes(x):
    if pd.notnull(x):
        ano = int(x.split(' ')[0])
        mes = int(x.split(' ')[3])
        return (ano*12)+mes
    else:
        return x

df_total['Credit_History_age'] = df_total['Credit_History_Age'].apply(lambda x: converter_mes(x)).astype(float)

In [ ]:
df_total['Type_of_Loan_ajustado'] = df_total['Type_of_Loan'].replace("[abc]* and ", " ", regex=True)

In [ ]:
# Esta função define a quantidade % de dados faltantes
def plot_nas(df: pd.DataFrame):
    if df.isnull().sum().sum() != 0:
        na_df = (df.isnull().sum() / len(df)) * 100      
        na_df = na_df.drop(na_df[na_df == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio %' :na_df})
        missing_data.plot(kind = "barh")
        plt.title("% de dados faltantes")
        plt.show()
    else:
        print('No NAs found')
        
plot_nas(df_total)

In [ ]:
df_total.info()

In [ ]:
#criando dicionario e convertendo os dados

dicionario_conversao = {
    'Age': int,
    'Num_Bank_Accounts': int,
    'Num_Credit_Card': int,
    'Num_of_Loan': int,
    'Annual_Income' : float,
    'Monthly_Inhand_Salary' : float,
    'Interest_Rate' : float,
    'Delay_from_due_date' : float,
    'Changed_Credit_Limit' : float,
    'Num_Credit_Inquiries' : float,
    'Outstanding_Debt' : float,
    'Credit_Utilization_Ratio' : float,
    'Amount_invested_monthly' : float,
    'Total_EMI_per_month' : float,
    'Num_of_Delayed_Payment' : float,
    'ID' : object,
    'Customer_ID' : object,
    'Name' : object,
    'Month' : object,
    'SSN' : object,
    'Type_of_Loan' : object,
    'Occupation' : object,
    'Credit_Mix' : object,
    'Payment_of_Min_Amount' : object,
    'Payment_Behaviour' : object,
    'test' : object
    }
# aplicando as type para variaveis

df_total = df_total.astype(dicionario_conversao)

In [ ]:
df_total.head()

### Data cleaning - Age

 * Limitando a idade dos clientes de 0 a 100

In [ ]:
df_total = df_total[(df_total['Age'] <= 100) & (df_total['Age'] >= 0)]

sns.boxplot(data=df_total, y='Age', x='test')

print(df_total.Age.describe())

### Concentração por tipo de ocupação

In [ ]:
sns.countplot(data=df_total, x = 'Occupation')
plt.xticks(rotation=90)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Age', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Idade x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Idade')
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Num_Bank_Accounts', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Idade x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Idade')
plt.show()
print(df_total.Num_Bank_Accounts.describe())

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Delay_from_due_date', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição dias de atraso x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('dias de atraso')
plt.show()
print(df_total.Delay_from_due_date.describe())

In [ ]:
df_total.columns

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Num_Credit_Card', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Qtd. Cartão de Crédito x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Qtd. Cartão de Crédito')
plt.show()
print(df_total.Num_Credit_Card.describe())

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Num_Bank_Accounts', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Qtd. Contas x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Qtd. Contas')
plt.show()
print(df_total.Num_Bank_Accounts.describe())

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Num_of_Loan', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Qtd. Empréstimos x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Qtd. Empréstimos')
plt.show()
print(df_total.Num_of_Loan.describe())

In [ ]:
"""
numCols = df_total.select_dtypes([np.number]).columns

for col in numCols:
    fig, ax = plt.subplots(1, 2, figsize = (8,8))
    sns.boxplot(data=df_total, y=col, x = 'Credit_Score', ax=ax[0]) #color = ['#f5190a', '#1e9e19', '#dede16'])
    #sns.scatterplot(data=df_total,x = 'Credit_Score', y = s = 100, y=col, ax=ax[1], color ='#ee1199')
    plt.show()
"""

In [ ]:
msno.bar(df_total)

In [ ]:
sns.factorplot('Credit_Score', col = 'Occupation', data = df_total, kind = 'count', col_wrap = 4)

In [ ]:
grid = sns.FacetGrid(df_total, col = 'Credit_Score')
grid.map(sns.distplot, 'Monthly_Inhand_Salary')

In [ ]:
label = df_total.Credit_Score.value_counts().index
label_count = df_total.Credit_Score.value_counts().values
plt.figure(figsize=(3,3))
plt.pie(data=df_total, x=label_count, labels=label, autopct='%.2f', shadow=True, radius=1.5)
plt.title("Credit Score Pie Chart")
plt.show()

In [ ]:
#plot(df_total,'Occupation','Credit_Score')

In [ ]:
#plot(df_total,'Age','Credit_Score')

In [ ]:
plt.figure(figsize=(17,8))
sns.heatmap(df_total.corr(),annot=True,cmap='viridis')

In [ ]:
plt.figure(figsize=(25,6))
sns.violinplot(x='Payment_Behaviour',y='Age',data=df_total, hue='Credit_Score', palette='rainbow')
plt.title("Violin Plot of Payment Behaviour by Age, Separated by Credit Score")

In [ ]:
plt.figure(figsize=(25,6))
sns.violinplot(x='Payment_Behaviour',y='Credit_Utilization_Ratio',data=df_total, hue='Credit_Score', palette='rainbow')
plt.title("Violin Plot of Payment Behaviour by Credit Utilization Ratio, Separated by Credit Score")

In [ ]:
plt.figure(figsize=(59,1))
sns.displot(data=df_total, x="Credit_Utilization_Ratio", kde=True)

In [ ]:
numCols = ['Monthly_Inhand_Salary', 'Delay_from_due_date', 'Credit_Utilization_Ratio']

for col in numCols:
    plt.figure(figsize=(180,6))
    sns.displot(x=col,data=df_total, hue='Credit_Score', palette=["#ff006e", "#83c5be", "#3a0ca3"])
    plt.show()

In [ ]:
#DataPrep at its finest pra visualizar tudo, tirar insights, as distribuições, missing data zeros, etc e etc
# create_report(df_total)

In [ ]:
# convertendo mes para encoding

import datetime

df_total['Month'] = df_total['Month'].apply(lambda x: datetime.datetime.strptime(x, '%B').month)

In [ ]:
# pegando indices

df2 = df_total.groupby(["Customer_ID"])["Month"].nlargest(1)

# list comprehension para pegar os indices

indice_final = [i[1] for i in df2.index.values]

# filtrando indices

df_total = df_total.loc[indice_final]


### Seleção de Features e Feature Importance

In [ ]:
#Dropar colunas inúteis (ids etc) e Changed Credit Limit e Credi History Age (vide heatmap)

#ExtraTreeClassifier para usar método feature_importances_

#Logistic regression do statsmodels para usar os testes de hipótese do método summary()

#Técnicas de feature selection da última aula sobre o tema

#Checar variância de cada feature
# pd.options.display.float_format = '{:.6f}'.format
# df_heart.apply(np.var).sort_values(ascending = False)[ : 8]

#Tentar usar o VIF (pode ser interessante ou inútil)

In [ ]:
# colunas para dropar

In [ ]:
## Modelo Statsmodels teste de hipótese

import statsmodels.api as sm

df2 = df_total.copy()
df2 = df2.drop(['ID', 'Customer_ID', 'Name', 'Age', 'SSN', 'Type_of_Loan', 'Type_of_Loan_ajustado', 'Credit_History_Age', 'Changed_Credit_Limit', 'Credit_Utilization_Ratio',
                'Monthly_Inhand_Salary', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 'Amount_invested_monthly', 'Monthly_Balance', 'Credit_History_age'], axis=1)

X_train = df2[df2['test'] == 0].drop(['Credit_Score', 'test'], axis = 1)
X_test = df2[df2['test'] == 1].drop(['Credit_Score', 'test'], axis = 1)

X_train = pd.get_dummies(X_train, columns=['Occupation', 'Month', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['Occupation', 'Month', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour'], drop_first=True)

from sklearn.preprocessing import OrdinalEncoder
ordenc = OrdinalEncoder()

y_train = df2[df2['test'] == 0]['Credit_Score']
y_test = df2[df2['test'] == 1]['Credit_Score']

y_train = ordenc.fit_transform(y_train.values.reshape(-1,1))
y_test = y_test.replace(np.NaN, 0)

# X_train = sm.add_constant(X_train)
# X_test = sm.add_constant(X_test)

model = sm.OLS(y_train, X_train)
non_reg_OLS = model.fit()
non_reg_OLS.summary()

In [ ]:
#VIF
def calc_vif(data):
    vif_df = pd.DataFrame(columns=['Var', 'VIF'])
    x_var_names = data.columns
    
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop(x_var_names[i])]
        r2 = sm.OLS(y, x).fit().rsquared
        vif = round(1/(1-r2),3)
        vif_df.loc[i] = [x_var_names[i], vif]
    return vif_df.sort_values(by='VIF',axis = 0, ascending=False, inplace=False)

calc_vif(X_train)

In [ ]:
plt.figure(figsize=(30,18))
sns.heatmap(X_train.corr(),annot=True,cmap='viridis')

In [ ]:
#ExtraTreeClassifier para feature_importances_

from sklearn.tree import ExtraTreeClassifier

xtc = ExtraTreeClassifier()
xtc.fit(X_train, y_train)
feat_importance = pd.Series(xtc.feature_importances_, index=X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(20,8))
feat_importance.plot(kind='bar')

##### - Inputação de Dados

In [ ]:
# escolher colunas para inputar e as formas de fazê-lo

### Pré-processamento para modelagem

In [ ]:
#Testar dados normalizados e/ou padronizados de formas diferentes (MinMax, Standard, Robust, sem normalizar, etc)

#Encoding (OneHot, Dummy, Label, Ordinary etc)



### Modelagem

In [ ]:
# modelos referência (random ou dummy classifier), base (random forest com plot, extratrees, XGBoost, rede neural simples) e ensembles (stacking de xgboost com )